# Dimensiones observables de calidad de mercado
---
En este notebook se presenta la metodología de preprocesamiento para una base de datos de la acción de Ecopetrol (ECOPETL). El procedimiento descrito estandariza la información para que esta sea utilizada para calcular distintos parámetros de calidad de mercado. 

Una vez las bases de datos hayan sido inicializadas, esta información puede ser interpretada por los métodos de visualización presentados en el notebook `Visualization.ipynb`

Este notebook está organizado de la siguiente forma:

 1. Manejo de datos
 
 2. Profundidad de mercado
 
 3. Buy-sell
 
 4. Price impact
 
 5. Ejemplo

## 1. Manejo de datos
---
En esta sección se definen dos funciones:

 * `GetDate()` - Retorna un DataFrame que contiene los días para los que tenemos datos
 * `StockPreprocessing()` - Estandariza los encabezados de la base de datos y comienza a calcular algunos de los parámetros de calidad de mercado que se quiere monitorear. Estos son:
  * Precio BID, precio ASK, precio medio y *quoted spread*.


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import dateutil.parser
from pandas import DataFrame

import warnings
warnings.filterwarnings('ignore')

import time

import dask.dataframe as dd
from dask import delayed

import os

In [2]:
# MEJORA PARA DEFINIR LOS DIAS
# stockdata["dia"] = pd.to_datetime(data['Datetime'], format='%Y:%M:%D').dt.date

In [2]:
# Funcion que genera un DataFrame con todos los dias (sin repetir) para los cuales se tienen datos

def GetDate(stockdata):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Data of various stocks
    
    Return:
    ------
    days: DataFrame - DataFrame of all days for which there is data
    '''
    
    days = pd.DatetimeIndex(stockdata.index).normalize()
    days = pd.DataFrame(days)
    days.index = stockdata.index
    days.columns = ['dia']
    
    return days.drop_duplicates(keep='first').dia

In [3]:
def StockPreprocessing(stockdata, stock_ticker):
    """
    Parameters:
    ------
    stockdata:
    DataFrame - Data of various stocks
    
    stock_tickers:
    List, string - Stock ticker strings
    """
    
    ##############################################################################3
    
    # Funcion que inicializa las columnas: 'nombre', 'date_time', 'tipo', 'precio', 'volumen',
    #                                      'BID', 'ASK', 'Mid_Price', 'Quoted_Spread'

    def stock_preprocessing(stockdata, stock_ticker):
        '''
        Parameters:
        ------
        stockdata:
        DataFrame - Data of various stocks
        
        stock_ticker:
        String - Ticker of the single stock we are interested in
        
        
        Return:
        ------
        stockdata:
        DataFrame - Data of stocks with the folloeing initialized columns: 
        nombre', 'date_time', 'tipo', 'precio', 'volumen', 'BID', 'ASK', 'Mid_Price', 'Quoted_Spread'
        '''
        
        stockname = stock_ticker + " CB Equity"
        
        # Se cambian los nombres de las columnas y se elimina lo demas
        stockdata         = stockdata[['name', 'times', 'type', 'value', 'size']]
        stockdata.columns = ['nombre','date_time','tipo','precio','volumen']      
        stockdata         = stockdata[['nombre','date_time','tipo','precio','volumen']]
        
        
        # Seleccionamos los datos segun la accion y el horario que nos interesan
        stockdata        = stockdata.loc[(stockdata["nombre"] == stockname)]
        stockdata.index  = stockdata.date_time
        stockdata        = stockdata.between_time('9:30','15:55')
        stockdata['dia'] = pd.DatetimeIndex(stockdata.date_time).normalize()
        
        # DataFrame de los dias para los cuales stockdata tiene datos
        days = GetDate(stockdata)
        
        # Lista donde se guardaran los resultados de los calculos diarios
        BA = []
        
        for i in days:
            
            # Sacamos los datos para un unico dia
            stockdailydata = stockdata[stockdata.dia == str(i)]
            
            # Inicializamos columnas de precios BID y ASK
            init_values = stockdailydata.precio.values
            d           = {'BID': init_values, 'ASK': init_values}
            BA_df       = pd.DataFrame(data=d)
            
            # Calculamos los valores de los precios BID y ASK
            bid       = stockdailydata['tipo'] == 'BID'
            ask       = stockdailydata['tipo'] == 'ASK'
            BA_df.BID = np.multiply(bid.values, stockdailydata.precio.values)
            BA_df.ASK = np.multiply(ask.values, stockdailydata.precio.values)
            
            # Para los precios iguales a cero se arrastra el valor anterior
            BA_df['BID'] = BA_df['BID'].replace(to_replace = 0, method = 'ffill').values
            BA_df['ASK'] = BA_df['ASK'].replace(to_replace = 0, method = 'ffill').values
            
            # Donde el BID sea menor o igual al precio ASK se pone un NaN
            BA_df = BA_df.where(BA_df.BID <= BA_df.ASK, np.nan)
            
            # Para los precios del principio del dia, debemos poner NaN
            cols = ['BID', 'ASK']
            BA_df[cols] = BA_df[cols].replace({0:np.nan, 0:np.nan})
            
            # Con los precios BID y ASK se calcula el Mid_Price y el Quoted_Spread
            BA_df['Mid_price']     = 0.5*(BA_df['BID'].values + BA_df['ASK'].values)
            BA_df['Quoted_Spread'] = (BA_df['ASK'].values - BA_df['BID'].values)/(BA_df.Mid_price.values)
            
            # Se guarda todo en el DataFrame BA
            BA.append(BA_df)
        
        # Se unifican los valores para todos los dias
        BA        = pd.concat(BA, axis=0)
        BA.index  = stockdata.index
        stockdata = pd.concat([stockdata, BA], axis=1)
        
        # Reordenamos las columnas
        stockdata = stockdata[["date_time", "dia", "nombre", "tipo", "precio", "volumen", "ASK", "BID", 
                               "Mid_price", "Quoted_Spread"]]
        stockdata = stockdata.drop(columns=["date_time"])
        
        return stockdata
    
    ##############################################################################3
    
    if (type(stock_ticker) == list):
        # Temporary list where we will save results
        temp = []
        
        # We apply the preprocessing procedure to our list of stocks
        for ticker in stock_ticker:
            
            # We preprocess the data individually by ticker
            temp.append(stock_preprocessing(stockdata, ticker))
        
        # Concatenate the results
        data = pd.concat(temp, sort=True)
        
        # Return a dataframe sorted by timestamp
        data = data.sort_index()
        data = data[["dia", "nombre", "tipo", "precio", "volumen", "ASK", "BID",
                     "Mid_price", "Quoted_Spread"]]
        return data
    
    else:
        return stock_preprocessing(stockdata, stock_ticker)

## 2. Profundidad de mercado
---
En esta sección utilizamos la librería Dask para paralelizar el cálculo de los siguientes parámetros: *i)* La profundidad BID; *ii)* La profundidad ASK; *iii)* La profundidad TRADE.

El cálculo de estas cantidades se hace de manera diaria. Para ello, se definen las siguientes funciones:

 * `sep_date()` - Esta función arroja una lista cuyas entradas corresponden a los datos de nuestra base para cada día.
 * `DailyDepth()` - Calcula las profundidades especificadas para cada entrada de `sep_date()`.
 * `StockDepth()` - Encapsula las dos funciones anteriores para calcular los parámetros a partir de todos los datos diarios.

In [4]:
def sep_date(stockdata):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Data of a single stock    
    
    Return:
    ------
    daily_dfs:
    List - A list whose entries are DataFrames of data in a particular day
    '''
    
    # Obtenemos los dias para los cuales tenemos datos
    days = GetDate(stockdata)
    
    # Creamos la lista donde guardaremos los datos correspondientes a un dia
    daily_dfs = []
    
    # Separamos stockdata por dias
    for i in days:
        daily_dfs.append(stockdata.loc[stockdata["dia"] == i])
        
    return daily_dfs

In [5]:
def StockDepth(stockdata, stock_ticker):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Preprocessed data of a several stocks
    
    stock_ticker:
    List - List of stock ticker strings
    
    Return:
    ------
    result:
    DataFrame - Results of the depth calculations
    '''
    
    ##############################################################################3
    
    def stock_depth(stockdata, stock_ticker):
        
        # Seleccionamos los datos de la accion que nos interesa
        stockname = stock_ticker + " CB Equity"
        stockdata = stockdata.loc[stockdata["nombre"]==stockname]
        
        # Obtenemos un arreglo de los dias para los cuales tenemos datos
        days = stockdata["dia"].drop_duplicates().values
        
        # Inicializamos la lista en donde guardaremos los resultados intermedios
        temp = []
        
        # Calculamos las profundidades diarias
        for day in days:
            
            # Seleccionamos los datos del dia que estamos evaluando
            df = stockdata.loc[stockdata["dia"]==day][["tipo", "precio", "volumen"]].copy()
            
            # Primeras condiciones - Ponemos el volumen de la orden segun su tipo
            a = np.where(df['tipo'] == 'ASK', df.volumen, 0)
            b = np.where(df['tipo'] == 'BID', df.volumen, 0)
            
            # Condicion - transacciones tipo TRADE de venta
            a_t = (np.where(df['tipo'] == 'TRADE', 
                            (np.where(df['tipo'].shift(1) == 'ASK', 
                                      (np.where(df['precio'] == df['precio'].shift(1), -df.volumen, 0)), 0)), 0))
            
            # Condicion - transacciones tipo TRADE de compra
            b_t = (np.where(df['tipo'] == 'TRADE', 
                            (np.where(df['tipo'].shift(1) == 'BID', 
                                      (np.where(df['precio'] == df['precio'].shift(1), -df.volumen, 0)), 0)), 0))
            
            # Realizamos la suma acumulada de las profundidades segun las condiciones
            df['ASK_depth'] = (np.where(a_t != 0, a_t, a)).cumsum()
            df['BID_depth'] = (np.where(b_t != 0, b_t, b)).cumsum()
            df['Depth']     = df['ASK_depth'] + df['BID_depth']
            df['log_depth'] = np.log(df['Depth'].values)
            
            # Eliminamos columnas redundantes
            df = df.drop(columns=["tipo", "precio", "volumen"])
            
            # Vamos guardando los resultados de cada dia
            temp.append(df)
            
        # Concatenamos los resultados obtenidos en un unico DataFrame
        temp = pd.concat(temp, sort=True)
        
        # Concatenamos los resultados obtenidos con los datos previos
        result = pd.concat([stockdata, temp], axis=1)
        
        # date_time es el indice del DataFrame, entonces tenerlo como otra columna es redundante
        #result = result.drop(columns=["date_time"])
        
        # Reordenamos las columnas
        result = result[["dia", "nombre", "tipo", "precio", "ASK", "BID", "Mid_price", "Quoted_Spread", "volumen", 
                         "ASK_depth", "BID_depth", "Depth", "log_depth"]]
        # Retornamos el resultado
        return result
        
    ##############################################################################3
    
    if type(stock_ticker) == list:
        
        # A list where we will store intermediate results is initialized
        temp = []
        
        # We calculate each stock's depth individually
        for ticker in stock_ticker:
            
            # A stock is selected
            stockname = ticker + " CB Equity"
            df = stockdata.loc[stockdata["nombre"]==stockname]
            
            # Stock results are appended
            temp.append(stock_depth(df, ticker))
    
        # Results are concatenated
        result = pd.concat(temp, sort=True)
        
        # Reordering the columns
        result = result[["dia", "nombre", "tipo", "precio", "ASK", "BID", "Mid_price", "Quoted_Spread", "volumen", 
                         "ASK_depth", "BID_depth", "Depth", "log_depth"]]
        
        return result.sort_index()
    
    else:
        
        return stock_depth(stockdata, stock_ticker)

## 3. Buy-sell
---
En esta sección calculamos la dirección de la transacción. Es decir, se calcula si la transacción es iniciada por el comprador o por el vendedor. Para ello, definimos la función `InitiatingParty()`. Este método funciona de la siguiente manera:
 * Se filtran los datos para tener únicamente transacciones tipo TRADE
 * Los precios que se ven se comparan con el precio medio:
  * Si el precio ofrecido es **mayor** que el precio medio, la transacción fue iniciada por un *comprador* ($D=+1$)
  * Si el precio ofrecido es **menor** que el precio medio, la transacción fue iniciada por un *vendedor* ($D=-1$)

In [6]:
def InitiatingParty(stockdata):
    '''
    Parameters:
    ------
    stockdata:
    DataFrame - Preprocessed data of the stock
    
    Return:
    ------
    x:
    DataFrame - DataFrame of TRADE quotes with the party that initiated the trade
    '''
    
    # Only TRADE operations are considered
    stockdata = stockdata.loc[stockdata["tipo"] == "TRADE"]
    
    # Values are filled according to the transaction's direction criteria
    stockdata["iniciado"] = np.where(stockdata.precio > stockdata.Mid_price, 1, 
                                     np.where(np.isnan(stockdata.Mid_price), np.nan, -1))
    
    return stockdata

## 4. Price impact
---

El *price impact* es un parámetro no-observable que cuantifica el efecto que tiene la actividad del mercado sobre el precio de un activo. En este notebook calculamos el impacto en el precio según Kyle, quien argumenta que el volumen de activos transados impacta linealmente su precio.

En este modelo, el cambio del precio de mercado por las órdenes de compra está dado por:

$$\Delta P_{i, t, d} = \beta_{i, d}^1 D_{i, t, d}V_{i, t, d} + \varepsilon_{i, t, d}$$

donde $\Delta P_{i, t, d}$ denota el cambio en el precio del activo $i$, en la transacción $t$ del día $d$. $D_{i, t, d}$ es la dirección de la transacción, es decir si la transacción fue iniciada por un comprador ($D = +1$) o por un vendedor ($D=-1$). 

$V_{i, t, d}$ es el volumen de la orden.

En esta sección calculamos el impacto de precio según Kyle 1985. 
 * `ImpactParameters()` - Calcula los parámetros necesarios para realizar la regresión de impacto de Kyle.
 * `KyleImpactRegression()` - Calcula el valor del impacto al precio.

In [7]:
from datetime import timedelta
from sklearn import linear_model as lm
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [10]:
"""def impact_params(stockdata):
    days = stockdata["dia"].drop_duplicates().values
    res  = []
    
    for day in days:
        stockdailydata = stockdata.loc[stockdata["dia"] == day]
        
        stockdailydata['delta_p']    = stockdailydata['precio'].diff()
        stockdailydata['order_flow'] = stockdailydata.volumen.values * stockdailydata.iniciado.values
        
        res.append(stockdailydata)
        
    res_df = pd.concat(res, axis=0)
    return res_df"""

In [8]:
def ImpactParameters(stockdata, stock_ticker):
    
    ##############################################################################3
    
    def impact_params(stockdata):
        days = stockdata["dia"].drop_duplicates().values
        res  = []
        
        for day in days:
            stockdailydata = stockdata.loc[stockdata["dia"] == day]
            
            stockdailydata['delta_p']    = stockdailydata['precio'].diff()
            stockdailydata['order_flow'] = stockdailydata.volumen.values * stockdailydata.iniciado.values
            
            res.append(stockdailydata)
        
        res_df = pd.concat(res, axis=0)
        return res_df
    
    ##############################################################################3
    
    if(type(stock_ticker) == list):
        
        temp = []
        
        for ticker in stock_ticker:
            
            stockname = ticker + " CB Equity"
            df = stockdata.loc[stockdata["nombre"]==stockname]
            temp.append(impact_params(df))
            
        result = pd.concat(temp, sort=True)
        
        return result.sort_index()
            
    else:
        stockname = stock_ticker + " CB Equity"
        df = stockdata.loc[stockdata["nombre"]==stockname]
        
        return impact_params(df)

In [40]:
def KyleImpactRegression(stockdata, stock_ticker):
    
    ##############################################################################3
    
    def kyle(stockdata):
        
        #days = GetDate(stockdata)#.drop_duplicates(keep='first').dia
        days = stockdata["dia"].drop_duplicates().values
        res = []
        
        for i in days:
            
            stockdailydata = stockdata[stockdata.dia == str(i)]
            
            x1 = stockdailydata.delta_p.values
            x1 = x1.reshape(-1, 1)
            
            x2 = stockdailydata.order_flow.values
            x2 = sm.add_constant(x2.reshape(-1, 1))
            
            result = sm.OLS(x1, x2, missing='drop').fit()
            
            coef = result.params[1]
            pvalue = result.pvalues[1]
            trades = len(stockdailydata)
            
            temp = [i, coef, pvalue, trades]
            res.append(temp)
        
        res = pd.DataFrame(res, columns=['dia', 'coef_regresion', 'p_value', 'trades'])
        res = res.set_index('dia')
        
        return res
    
    ##############################################################################3
    
    if(type(stock_ticker)==list):
        
        result = dict()
        
        for ticker in stock_ticker:
            
            stockname = ticker + " CB Equity"
            df        = stockdata.loc[stockdata["nombre"]==stockname]
            
            print(stockname)
            #print(stockdata.head())
            
            result[ticker] = kyle(df)
            #print(result[ticker].head())
        
        return result
    
    else:
        
        stockname = stock_ticker + " CB Equity"
        stockdata = stockdata.loc[stockdata["nombre"]==stockname]
        
        return kyle(stockdata)

## Ejemplo
---
Ahora aplicamos las funciones definidas en las secciones previas:

**1. Manejo de datos**

In [10]:
# Establecemos la ruta del directorio para importar los datos
pathData = os.getcwd() + "/datos completos.csv"

In [11]:
# Importamos los datos
data = %time pd.read_csv(pathData, parse_dates=[2], sep=',', na_values='NA',low_memory=False)

CPU times: user 3.17 s, sys: 171 ms, total: 3.34 s
Wall time: 3.36 s


In [12]:
# Preprocesamos los datos
prep_data = %time StockPreprocessing(data, ['ECOPETL', 'BCOLO'])

CPU times: user 2.6 s, sys: 46.7 ms, total: 2.65 s
Wall time: 2.65 s


In [13]:
#prep_data.head(35)
prep_data.head()

,dia,nombre,tipo,precio,volumen,ASK,BID,Mid_price,Quoted_Spread
date_time,,,,,,,,,
2017-03-03 09:30:00,2017-03-03,ECOPETL CB Equity,ASK,1320.0,360000.0,1320.0,NaN,NaN,NaN
2017-03-03 09:30:00,2017-03-03,ECOPETL CB Equity,BID,1305.0,100000.0,1320.0,1305.0,1312.5,0.011429
2017-03-03 09:30:02,2017-03-03,ECOPETL CB Equity,ASK,1315.0,80000.0,1315.0,1305.0,1310.0,0.007634
2017-03-03 09:30:03,2017-03-03,ECOPETL CB Equity,ASK,1315.0,155240.0,1315.0,1305.0,1310.0,0.007634
2017-03-03 09:30:12,2017-03-03,ECOPETL CB Equity,BID,1305.0,170000.0,1315.0,1305.0,1310.0,0.007634


In [14]:
test = InitiatingParty(prep_data)
#test.head(15)
test.head()

,dia,nombre,tipo,precio,volumen,ASK,BID,Mid_price,Quoted_Spread,iniciado
date_time,,,,,,,,,,
2017-03-03 09:30:14,2017-03-03,ECOPETL CB Equity,TRADE,1315.0,2000.0,1315.0,1305.0,1310.0,0.007634,1.0
2017-03-03 09:30:22,2017-03-03,ECOPETL CB Equity,TRADE,1305.0,1000.0,1315.0,1305.0,1310.0,0.007634,-1.0
2017-03-03 09:30:40,2017-03-03,ECOPETL CB Equity,TRADE,1305.0,1351.0,1315.0,1305.0,1310.0,0.007634,-1.0
2017-03-03 09:31:00,2017-03-03,ECOPETL CB Equity,TRADE,1305.0,500.0,1315.0,1305.0,1310.0,0.007634,-1.0
2017-03-03 09:31:00,2017-03-03,ECOPETL CB Equity,TRADE,1305.0,500.0,1315.0,1305.0,1310.0,0.007634,-1.0


**2. Profundidad de mercado**

In [15]:
# Calculamos las profundidaes BID, ASK y TRADE

res = %time StockDepth(prep_data, ["ECOPETL", "BCOLO"])

CPU times: user 2.25 s, sys: 22.2 ms, total: 2.27 s
Wall time: 2.35 s


In [18]:
# Miramos los resultados
#res.head(12)
res.head()

,dia,nombre,tipo,precio,ASK,BID,Mid_price,Quoted_Spread,volumen,ASK_depth,BID_depth,Depth,log_depth
date_time,,,,,,,,,,,,,
2017-03-03 09:30:00,2017-03-03,ECOPETL CB Equity,ASK,1320.0,1320.0,NaN,NaN,NaN,360000.0,360000.0,0.0,360000.0,12.793859
2017-03-03 09:30:00,2017-03-03,ECOPETL CB Equity,BID,1305.0,1320.0,1305.0,1312.5,0.011429,100000.0,360000.0,100000.0,460000.0,13.038982
2017-03-03 09:30:02,2017-03-03,ECOPETL CB Equity,ASK,1315.0,1315.0,1305.0,1310.0,0.007634,80000.0,440000.0,100000.0,540000.0,13.199324
2017-03-03 09:30:03,2017-03-03,ECOPETL CB Equity,ASK,1315.0,1315.0,1305.0,1310.0,0.007634,155240.0,595240.0,100000.0,695240.0,13.452012
2017-03-03 09:30:12,2017-03-03,ECOPETL CB Equity,BID,1305.0,1315.0,1305.0,1310.0,0.007634,170000.0,595240.0,270000.0,865240.0,13.670762


In [16]:
# Guardamos los resultados
#res.to_csv("depth_data.csv")

**3. Buy-sell**

In [19]:
# Encontramos la partida que inicia la transaccion
init_trans = InitiatingParty(prep_data)

In [21]:
# Miramos los resultados
init_trans.head()

,dia,nombre,tipo,precio,volumen,ASK,BID,Mid_price,Quoted_Spread,iniciado
date_time,,,,,,,,,,
2017-03-03 09:30:14,2017-03-03,ECOPETL CB Equity,TRADE,1315.0,2000.0,1315.0,1305.0,1310.0,0.007634,1.0
2017-03-03 09:30:22,2017-03-03,ECOPETL CB Equity,TRADE,1305.0,1000.0,1315.0,1305.0,1310.0,0.007634,-1.0
2017-03-03 09:30:40,2017-03-03,ECOPETL CB Equity,TRADE,1305.0,1351.0,1315.0,1305.0,1310.0,0.007634,-1.0
2017-03-03 09:31:00,2017-03-03,ECOPETL CB Equity,TRADE,1305.0,500.0,1315.0,1305.0,1310.0,0.007634,-1.0
2017-03-03 09:31:00,2017-03-03,ECOPETL CB Equity,TRADE,1305.0,500.0,1315.0,1305.0,1310.0,0.007634,-1.0


**4. Price impact**

In [32]:
# Calculamos los parametros de impacto
impact_params = ImpactParameters(init_trans, ["ECOPETL", "BCOLO"])

# Miramos los resultados
impact_params.head()

,ASK,BID,Mid_price,Quoted_Spread,delta_p,dia,iniciado,nombre,order_flow,precio,tipo,volumen
date_time,,,,,,,,,,,,
2017-03-03 09:30:14,1315.0,1305.0,1310.0,0.007634,NaN,2017-03-03,1.0,ECOPETL CB Equity,2000.0,1315.0,TRADE,2000.0
2017-03-03 09:30:22,1315.0,1305.0,1310.0,0.007634,-10.0,2017-03-03,-1.0,ECOPETL CB Equity,-1000.0,1305.0,TRADE,1000.0
2017-03-03 09:30:40,1315.0,1305.0,1310.0,0.007634,0.0,2017-03-03,-1.0,ECOPETL CB Equity,-1351.0,1305.0,TRADE,1351.0
2017-03-03 09:31:00,1315.0,1305.0,1310.0,0.007634,0.0,2017-03-03,-1.0,ECOPETL CB Equity,-500.0,1305.0,TRADE,500.0
2017-03-03 09:31:00,1315.0,1305.0,1310.0,0.007634,0.0,2017-03-03,-1.0,ECOPETL CB Equity,-500.0,1305.0,TRADE,500.0


In [33]:
impact_params.loc[impact_params["nombre"]=="BCOLO CB Equity"].head()

,ASK,BID,Mid_price,Quoted_Spread,delta_p,dia,iniciado,nombre,order_flow,precio,tipo,volumen
date_time,,,,,,,,,,,,
2017-03-03 09:33:29,25040.0,24920.0,24980.0,0.004804,NaN,2017-03-03,1.0,BCOLO CB Equity,5520.0,25040.0,TRADE,5520.0
2017-03-03 10:01:20,25040.0,24980.0,25010.0,0.002399,0.0,2017-03-03,1.0,BCOLO CB Equity,436.0,25040.0,TRADE,436.0
2017-03-03 10:07:14,25240.0,25040.0,25140.0,0.007955,0.0,2017-03-03,-1.0,BCOLO CB Equity,-2277.0,25040.0,TRADE,2277.0
2017-03-03 10:07:14,25240.0,25040.0,25140.0,0.007955,0.0,2017-03-03,-1.0,BCOLO CB Equity,-759.0,25040.0,TRADE,759.0
2017-03-03 10:07:14,25240.0,25040.0,25140.0,0.007955,0.0,2017-03-03,-1.0,BCOLO CB Equity,-86.0,25040.0,TRADE,86.0


In [41]:
# Calculamos el coeficiente de impacto
kyle_reg = KyleImpactRegression(impact_params, ["ECOPETL", "BCOLO"])

# Miramos los resultados
#kyle_reg.head()

ECOPETL CB Equity
BCOLO CB Equity


In [42]:
kyle_reg.keys()

dict_keys(['ECOPETL', 'BCOLO'])

In [43]:
kyle_reg["BCOLO"]

,coef_regresion,p_value,trades
dia,,,
2017-03-03,0.000117,9.594568e-01,48
2017-03-06,-0.009953,4.689258e-03,27
2017-03-07,0.003211,1.775236e-05,69
2017-03-08,0.001706,4.931351e-01,43
2017-03-09,0.000456,8.726663e-01,36
2017-03-10,-0.000140,2.657689e-01,46
2017-03-13,-0.001183,5.618614e-01,54
2017-03-14,-0.000552,4.602736e-01,152
2017-03-15,-0.004808,6.056292e-02,39
